# ETL: Extract, Transform and Load Process

Utilizando a API de dados de ações Alpha Vantage, nós vamos criar uma nova base de dados com as ações, informando os dividendos dos últimos 5 anos. 

## Extract: Coletando os dados

Os ativos da bolsa americana, incluindo as ações das empresas (Stocks), estão em um arquivo .csv. Ao carregá-los em um dataframe, estamos na etapa de extração.

In [2]:
import pandas as pd

df = pd.read_csv('listing_status.csv', header=0)
df.head(10)

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active
5,AACT,Ares Acquisition Corporation II - Class A,NYSE,Stock,2023-06-12,NaN,Active
6,AACT-U,Ares Acquisition Corporation II - Units (1 Ord...,NYSE,Stock,2023-04-21,NaN,Active
7,AACT-WS,Ares Acquisition Corporation II - Warrants (01...,NYSE,Stock,2023-06-12,NaN,Active
8,AADI,Aadi Bioscience Inc,NASDAQ,Stock,2017-08-08,NaN,Active
9,AADR,ADVISORSHARES DORSEY WRIGHT ADR ETF,NASDAQ,ETF,2010-07-21,NaN,Active


Vimos que a coluna `delistingDate` contém um dado inválido (NaN - Not a Number). Podemos conferir os tipos de dados para avaliar melhor essa e outras inconsistências. 

In [3]:
df.dtypes

symbol            object
name              object
exchange          object
assetType         object
ipoDate           object
delistingDate    float64
status            object
dtype: object

Veja que as colunas de data tem um tipo `object` ou `float64`, quando a coluna deveria trazer um tipo de data. Para a coluna `ipoDate` (data de entrada na bolsa de valores), o tipo não é consistente para exportar para uma outra base de dados ou realizar comparações. Para a coluna `delistingDate` o caso é ainda mais grave: se pensarmos em um dataframe muito grande com esse tipo de inconsistência, o espaço ocupado por cada valor inválido ocupa 64 bits, o que pode fazer que esse desperdício cresca de forma exponencial.

Vamos avaliar, portanto, quantas e quais colunas tem valores nulos:

In [6]:
df.isna().sum()

symbol               1
name                34
exchange             0
assetType            0
ipoDate              0
delistingDate    11628
status               0
dtype: int64

Em seguida, vamos comparar com o tamanho do dataframe:

In [7]:
df.shape

(11628, 7)

Nós podemos tratar desse problema de várias maneiras. Vejamos a etapa de transformação:

## Transformation: Limpando os dados

Na verificação de valores nulos, vimos que há 34 ocorrências para a coluna `name` e 1 para a coluna `symbol` . 

Você pode tratar isso de duas maneiras: corrigir esses dados ou simplesmente excluir essas linhas. Optaremos pela segunda opção, para fins de simplificação deste tutorial. Porém, caso você queira, é possível realizar um novo ET(L) para recuperar os nomes das empresas e preencher no dataframe, bem como o símbolo vazio, usando outra API de dados financeiros, como o Google Finance, por exemplo.

In [8]:
df[df.name.isna()]

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
456,AMEH,NaN,NASDAQ,Stock,2024-02-26,NaN,Active
487,AMRS,NaN,NASDAQ,Stock,2023-08-18,NaN,Active
896,AVRO,NaN,NASDAQ,Stock,2024-06-21,NaN,Active
2113,CLVS,NaN,NASDAQ,Stock,2023-01-03,NaN,Active
2419,CTEST,NaN,NYSE,Stock,2019-07-25,NaN,Active
2632,DEC,NaN,NYSE,Stock,2023-12-18,NaN,Active
2676,DFFN,NaN,NASDAQ,Stock,2023-08-17,NaN,Active
4135,FWP,NaN,NASDAQ,Stock,2022-12-27,NaN,Active
6994,MTEST,NaN,NYSE,Stock,2019-10-09,NaN,Active
7401,NTEST-G,NaN,NYSE,Stock,2019-07-17,NaN,Active


In [9]:
df.dropna(subset=['name', 'symbol'], inplace=True)

In [10]:
df.shape

(11593, 7)

## Transformation: Normalizando os dados

Como você pode perceber, toda a coluna `delistingDate` possui valores nulos, pois o número de ocorrências é igual ao número de linhas do dataframe. Logo, a depender do propósito da sua análise, é possível corrigir de várias maneiras: preencher com valores de string, excluir a coluna, caso não seja necessário, ou corrigir o tipo de dado desta coluna, que é exatamente o que vamos optar por fazer no código a seguir, aplicando também para a coluna `ipoDate`. 

In [11]:
df['ipoDate'] = pd.to_datetime(df['ipoDate'], format='%Y-%m-%d')
df['delistingDate'] = pd.to_datetime(df['delistingDate'], format='%Y-%m-%d')

Vejamos como ficaram os tipos de informação e como ficou o dataframe corrigido, com NaT (Not a Date) na coluna `delistingDate`:

In [12]:
df.dtypes

symbol                   object
name                     object
exchange                 object
assetType                object
ipoDate          datetime64[ns]
delistingDate    datetime64[ns]
status                   object
dtype: object

In [13]:
df.head()

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaT,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaT,Active
2,AAA,ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaT,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaT,Active
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaT,Active


## Transformation: Filtrando os dados

Para cumprir com o propósito da nossa análise, é necessário retirar todos os dados que não são ações. Antes, vamos conferir quais são os tipos de informações na coluna `assetType`:

In [14]:
df.assetType.unique()

array(['Stock', 'ETF'], dtype=object)

Então, vamos criar uma cópia desse dataframe só com as Stocks, e depois retirar essa coluna do novo dataframe, que não será mais necessária:

In [15]:
stocks = df[df.assetType == 'Stock']
stocks = stocks.drop('assetType', axis=1)
stocks.reset_index(drop=True, inplace=True)
stocks.head()

,symbol,name,exchange,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,1999-11-18,NaT,Active
1,AA,Alcoa Corp,NYSE,2016-10-18,NaT,Active
2,AACG,ATA Creativity Global,NASDAQ,2008-01-29,NaT,Active
3,AACT,Ares Acquisition Corporation II - Class A,NYSE,2023-06-12,NaT,Active
4,AACT-U,Ares Acquisition Corporation II - Units (1 Ord...,NYSE,2023-04-21,NaT,Active


A partir de agora vamos fazer outra aquisição de dados, com o uso da API Alpha Vantage, de todos os dividendos pagos das ações. Essa é uma nova extração de dados:

In [18]:
import requests

apikey = 'YFKTH7JD4KOJVNAY'

url = f'https://www.alphavantage.co/query?function=DIVIDENDS&symbol=IBM&apikey={apikey}'
response = requests.get(url)
data = response.json()

# transform data into dataframe
ibm = pd.DataFrame(data['data'])

ibm

,ex_dividend_date,declaration_date,record_date,payment_date,amount
0,2025-02-10,2025-01-28,2025-02-10,2025-03-10,1.67
1,2024-11-12,2024-10-30,2024-11-12,2024-12-10,1.67
2,2024-08-09,2024-07-29,2024-08-09,2024-09-10,1.67
3,2024-05-09,2024-04-30,2024-05-10,2024-06-10,1.67
4,2024-02-08,2024-01-30,2024-02-09,2024-03-09,1.66
...,...,...,...,...,...
100,2000-02-08,None,None,None,0.12
101,1999-11-08,None,None,None,0.12
102,1999-08-06,None,None,None,0.12
103,1999-05-06,None,None,None,0.24


Para conferência, analisemos os tipos de dados do dataframe:

In [19]:
ibm.dtypes

ex_dividend_date    object
declaration_date    object
record_date         object
payment_date        object
amount              object
dtype: object

Vamos definir uma função que faz a extração e a transformação dos dados, afinal, para comparação, as colunas de data precisam ser do tipo data e o amount precisa ser do tipo float. Para fins de registro, também faremos a exclusão das linhas que não contém todas as informações de datas dos dividendos:

In [20]:
def extract_dividends_df(symbol, year):
    url = f'https://www.alphavantage.co/query?function=DIVIDENDS&symbol={symbol}&apikey={apikey}'
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data['data'])
    df = df[df['ex_dividend_date'] > f'{year}-12-31']
    
    df['ex_dividend_date'] = pd.to_datetime(df['ex_dividend_date'], format='%Y-%m-%d', errors='coerce')
    df['declaration_date'] = pd.to_datetime(df['declaration_date'], format='%Y-%m-%d', errors='coerce')
    df['record_date'] = pd.to_datetime(df['record_date'], format='%Y-%m-%d', errors='coerce')
    df['payment_date'] = pd.to_datetime(df['payment_date'], format='%Y-%m-%d', errors='coerce')
    
    df = df.astype({'amount': 'float64'})
    return df

Vejamos outro exemplo de empresa:

In [21]:
apple = extract_dividends_df('AAPL', 2019)
apple


,ex_dividend_date,declaration_date,record_date,payment_date,amount
0,2025-02-10,2025-01-30,2025-02-10,2025-02-13,0.250
1,2024-11-08,2024-10-31,2024-11-11,2024-11-14,0.250
2,2024-08-12,2024-08-01,2024-08-12,2024-08-15,0.250
3,2024-05-10,2024-05-02,2024-05-13,2024-05-16,0.250
4,2024-02-09,2024-02-01,2024-02-12,2024-02-15,0.240
5,2023-11-10,2023-11-02,2023-11-13,2023-11-16,0.240
6,2023-08-11,NaT,NaT,NaT,0.240
7,2023-05-12,2023-05-04,2023-05-15,2023-05-18,0.240
8,2023-02-10,2023-02-02,2023-02-13,2023-02-16,0.230
9,2022-12-23,2022-12-19,2022-12-28,2023-01-09,0.000


In [22]:
apple.dtypes

ex_dividend_date    datetime64[ns]
declaration_date    datetime64[ns]
record_date         datetime64[ns]
payment_date        datetime64[ns]
amount                     float64
dtype: object

## Transformation: Combinação de dados

Faremos a junção, para cada uma das ações, de uma nova coluna, com a informação da soma dos dividendos pagos nos últimos 5 anos. 

Como a API restringe o número de requisições por dia (máx. 25), então teremos que fazer individualmente para algumas ações. Deixaremos também o exemplo se caso pudéssemos fazer de todas (se pagássemos o plano Premium dessa API):

In [23]:
# para cada ação, somar todos os dividendos dos últimos 5 anos
# stocks['dividends_last_5_years'] = stocks.symbol.apply(lambda x: extract_dividends_df(x, 2019).amount.sum())

stocks['dividends_last_5_years'] = 0.0
stocks.loc[stocks.symbol == 'AAPL', 'dividends_last_5_years'] = apple.amount.sum()
stocks[stocks.symbol == 'AAPL']

,symbol,name,exchange,ipoDate,delistingDate,status,dividends_last_5_years
17,AAPL,Apple Inc,NASDAQ,1980-12-12,NaT,Active,6.58


In [24]:
ibm = extract_dividends_df('IBM', 2019)
stocks.loc[stocks.symbol == 'IBM', 'dividends_last_5_years'] = ibm.amount.sum()
stocks[stocks.symbol == 'IBM']

,symbol,name,exchange,ipoDate,delistingDate,status,dividends_last_5_years
3350,IBM,International Business Machines Corp,NYSE,1962-01-02,NaT,Active,34.62


## Load: Carregando as novas informações em um novo banco de dados

Podemos utilizar o que aprendemos na aula de banco de dados e guardar esse dataframe em uma nova tabela:

In [25]:
import sqlite3

conn = sqlite3.connect('coderhouse.db')
stocks.to_sql('stocks', conn, if_exists='replace', index=False)
conn.close()

## Sugestão de novas features

Você pode comparar o desempenho dos dividendos de acordo entre as bolsas de valores (NASDAQ e NYSE):

In [26]:
stocks.exchange.unique()

array(['NYSE', 'NASDAQ', 'NYSE MKT', 'NYSE ARCA', 'BATS'], dtype=object)